Half of this lecture is in a google colab notebook cause I'm a moron. Find it here: <https://colab.research.google.com/drive/153aLhRFqSwJ70xoTRJWHsi-9WFjkHfdQ#scrollTo=Myk8rHCi2T8D&uniqifier=5>

# Combining Objects: Composition

The instance variables defined in a class and stored in the objects of that class can themselves be objects. We can make lists of objects, tuples of objects, etc.

Often we will want to create a new class with instance variables that are objects created from classes that we have previously created. For example, if we create a new class Rect to represent rectangles, we might want to use Point objects to represent two corners of the rectangle:

In [4]:
class Point:
    """An (x,y) coordinate pair"""
    def __init__(self, x, y):
        self.x = x
        self.y = y
    
    def move(self, d: "Point") -> "Point":
        """(x,y).move(dx,dy) = (x+dx, y+dy)"""
        x = self.x + d.x
        y = self.y + d.y
        return Point(x,y)
        
    def move_to(self, new_x, new_y):
        """Change the coordinates of this Point"""
        self.x = new_x
        self.y = new_y
        
    def __add__(self, other: "Point"):
        """(x,y) + (dx, dy) = (x+dx, y+dy)"""
        return Point(self.x + other.x, self.y + other.y)

    def __str__(self) -> str:
        """Printed representation.
        str(p) is an implicit call to p.__str__()
        """
        return f"({self.x}, {self.y})"
      
    def __repr__(self) -> str:
        """Debugging representation.  This is what
        we see if we type a point name at the console.
        """
        return f"Point({self.x}, {self.y})"

In [6]:
class Rect:
    """A rectangle is represented by a pair of points
    (x_min, y_min), (x_max, y_max) at opposite corners.
    Whether (x_min, y_min) is lower left or upper left
    depends on the coordinate system.
    """
    def __init__(self, xy_min: Point, xy_max: Point):
        self.min_pt = xy_min
        self.max_pt = xy_max
    
    def area(self):
        """Area is height * width"""
        height = self.max_pt.x - self.min_pt.x
        width = self.max_pt.y - self.min_pt.y
        return height * width

    def translate(self, delta: Point) -> "Rect":
        """New rectangle offset from this one by delta as movement vector"""
        return Rect(self.min_pt + delta, self.max_pt + delta)

    def __repr__(self) -> str:
        return f"Rect({repr(self.min_pt)}, {repr(self.max_pt)}"

    def __str__(self) -> str:
        return f"Rect({self.min_pt}, {self.max_pt})"

p1 = Point(3,5)
p2 = Point(8,7)
r1 = Rect(p1, p2)

mvmt = Point(4, 5)
r2 = r1.translate(mvmt) # r2 is a new rectangle, not a copy of r1 - treate Point (4,5) as (dx, dy)

print(f'{r1} + {mvmt} -> {r2}')
print(f'Area of {r1} is {r1.area()}')

Rect((3, 5), (8, 7)) + (4, 5) -> Rect((7, 10), (12, 12))
Area of Rect((3, 5), (8, 7)) is 10


Note that the height and width are local variables that exist only while method area is executing. min_pt and max_pt, on the other hand, are instance variables that are stored within the Rect object.

Suppose we ran the above code in PythonTutor. (PythonTutor cannot import Number, but for the examples we could replace it with int.) What picture would it draw of r1? Would height and width in method area be included as instance variables? Why or why not?

# Wrapping and Delegation
Sometimes we want a class of objects that is almost like an existing class, but with a little extra information or a few new methods. One way to do this is to build a new class that wraps an existing class, often a built-in class like list or dict. (In the next chapter we will see another approach.)

Suppose we wanted objects that provide some of the same functionality as list objects, and also some new functionality or some restrictions. For example, we might want a method area that returns the sum of the areas of all the Rect objects in the RectList:

In [10]:
class RectList:
    """A collection of Rects."""

    def __init__(self):
        self.elements = [ ]

    def area(self):
        total = 0
        for el in self.elements:
            total += el.area()
        return total

That seems reasonable, but how do we add Rect objects to the Rectlist?

We do not want to do it this way:

In [11]:
li = RectList()
# DON'T DO THIS
li.elements.append(Rect(Point(3,3), Point(5,7)))
li.elements.append(Rect(Point(2,2), Point(3,3)))

As a general rule, we should be cautious about accessing the instance variables of an object outside of methods of the object’s class, and we should especially avoid modifying instance variables anywhere except in methods. Code that “breaks the abstraction”, like the example above calling the append method of the elements instance variable, is difficult to read and maintain. 

So we want instead to give RectList it’s own append method, so that we can write

In [14]:
class RectList:
    """A collection of Rects."""

    def __init__(self):
        self.elements = [ ]
        
    def append(self, item: Rect):
        """Delegate to elements"""
        self.elements.append(item)

    def area(self):
        total = 0
        for el in self.elements:
            total += el.area()
        return total

In [15]:
li = RectList()
li.append(Rect(Point(3,3), Point(5,7)))
li.append(Rect(Point(2,2), Point(3,3)))
print(f"Combined area is {li.area()}")

Combined area is 9


We call this delegation because append method of RectList method
just hands off the work to the append method of class list. When we write a wrapper class, we typically write several such trivial delegation methods.

Wrapping and delegation work well when we want the wrapper class (like RectList in this example) to have a few of the same methods as the wrapped class (list). When we want the new collection class to have all or nearly all the methods of an existing collection, the **inheritance approach** introduced in the next chapter is more appropriate.

# DSA - What are Classes and Objects? From CodingDojo

two  robots-  tom and jerry

tom - red, 30lbs
jerry - blue, 40lbs

How do we represent these entities?

Store two sets of information for each robot
name, color, weight - 3 variables

+ function introduceself()

organizing properties and functions together creates an object.

once yoy have an object, you can create multiple instances of it, and put it in a variable: say all the information for the tom object is in r1.

Once you create the object for jerry, assigning his color, name, and weight, you can put it in a variable r2.

functions in an object are called methods. Variables in in object are called attributes.

# So what the hell is a class?

a class is basically a template from which you use to create objects.

When you make an object,you'll want to know what attributes and methods it has. You can make a class for this.

## Class

-  Classes don't refer to any particular object-  they are just a template for creating objects.
-  Classes are like blueprints for creating objects.

- Classes do contain the methods and attributes that objects will have.

- Classes are like factories that produce objects.

- Classes have to have names - And the first letter of their name should be capitalized.

In [16]:
class Robot:
    def __init__(self, name, weight, color):
        self.name = name
        self.weight = weight
        self.color = color

    def introduce(self):
        print(f"Hi, I'm {self.name} and I weigh {self.weight} pounds and I'm {self.color}")
    
#create instances of the class - objects
r1 = Robot("Tom", 30, "red") #this is an object!
r2 = Robot("Jerry", 40, "blue") #this is an object!

#Access attributes and methods of the objects
print(r1.name)
print(r2.color)
print(r1.introduce())


Tom
blue
Hi, I'm Tom and I weigh 30 pounds and I'm red
None


In the example above, we define the Robot class with an __init__ method (constructor) that takes three arguments: name, weight, and color. The self parameter represents the instance of the class being created. Inside the constructor, we initialize the instance variables name, weight, and color with the values passed as arguments.

Additionally, we define a method called introduce within the class, which allows the robot to introduce itself with its attributes. The method uses the instance variables (attributes) name, weight, and color to create a formatted introduction.

We then create two instances of the Robot class, robot1 and robot2, and access their attributes and methods using dot notation.

# What's a Constructor?

A constructor is a special method that is used to initialize a newly created object and is called just after the memory is allocated for the object. It can be used to initialize the variables of the object to either default values or user-defined values.

This is a constructor (in python):

```python

def __init__(self, name, weight, color):
        self.name = name
        self.weight = weight
        self.color = color

```

# How do Multiple classes and Objects Interact?

**Composition and Aggregation**
 - Composition is when one class contains another class as a part. The contained class cannot exist without the class that contains it. When an object of the outer class is created, an object of the inner class is also created within it. The outer class controls the inner class. For example, a car has an engine. Without an engine, a car cannot function. The engine cannot be used outside of the car. The car controls the engine. This is an example of composition.

 - Aggregation is similar but with a weaker relation: the contained class can exist without the class that contains it. For example, a car has a radio. The radio can exist outside of the car. The car does not control the radio. However, the radio is part of the car. This is an example of aggregation.

**Inheritance**
-  The process where one class (the subclass or derived class) inherits attributes and methods from another class (the superclass or base class). Inheritance is used to define a new class based on an existing class. The new class inherits all the attributes and methods of the existing class but can also define additional attributes and methods and override existing methods.

-  This allows you to reuse existing code and to reduce the complexity of programs. Inheritance is also known as subclassing. You can create Heirarchies of classes, where a class inherits from another class, which in turn inherits from another class.

- Lets you create heirarchies and "Is-a Relationships", where a 

**Method calls**

- Objects of one class can call methods of other class. This is useful for making one class do some work and then pass the result to another class for further processing. Or make one class interact with another to perform specific tasks or share information.

**Passing objects as arguments to functions**
- Objects of one class can be passed as arguements to methods of another class, allowing them to collaborate and exchange data.

**Accessing Attributes**
- Objects of one class can access attributes of another class. This allows them to share data and read/modify values.

**Event handling**
- Objects can emit events or signals, and other objects can listen for these events and respond to them. This allows objects to communicate with each other and respond to changes in the system.

# Here are two classes interacting using composition:

In [17]:
class Engine:
    def __init__(self, horsepower):
        self.horsepower = horsepower

    def start(self):
        return "Engine started."

    def stop(self):
        return "Engine stopped."


class Car:
    def __init__(self, make, model, engine):
        self.make = make
        self.model = model
        self.engine = engine

    def start(self):
        return f"{self.make} {self.model}: {self.engine.start()}"

    def stop(self):
        return f"{self.make} {self.model}: {self.engine.stop()}"


# Create instances of Engine and Car classes
engine = Engine(horsepower=200)
car = Car(make="Toyota", model="Corolla", engine=engine)

# Interact with objects
print(car.start())  # Output: Toyota Corolla: Engine started.
print(car.stop())   # Output: Toyota Corolla: Engine stopped.


Toyota Corolla: Engine started.
Toyota Corolla: Engine stopped.


In [13]:
import random

class Person:
    def __init__(self, name, is_sitting, robot_owned):
        self.name = name
        self.personality = "nice"
        self.is_sitting = is_sitting
        self.robot_owned = robot_owned

    def __personality__(self):   
        for i in range(0, 5):
                i = random.randint(0, 4)
                if i == 0:
                    self.personality = "nice"
                elif i == 1:
                    self.personality = "mean"
                elif i == 2:
                    self.personality = "funny"
                elif i == 3:
                    self.personality = "smart"
                elif i == 4:
                    self.personality = "dumb"
            
        return f"{self.name} is {self.personality}."


    def __is_sitting__(self):
        if self.is_sitting == True:
            return f"{self.name} is sitting."
        
        else:
            return f"{self.name} is not sitting."
    
    def __robot_owned__(self):
        if self.robot_owned == True:
            return f"{self.name} owns a robot."
        
        else:
            return f"{self.name} does not own a robot."
        
john = Person("John", True, True)

print(f'{john.__is_sitting__()} {john.__robot_owned__()}')

john.is_sitting = False
john.robot_owned = False

print(f'{john.__is_sitting__()} {john.__robot_owned__()}')
print(john.__personality__())

John is sitting. John owns a robot.
John is not sitting. John does not own a robot.
John is smart.


# Classes and Objects in Python Part Two

## How multiple classes and objects interact with each other in python.



In [14]:
class Robot:
    def __init__(self, name, color, weight):
        self.name = name
        self.color = color
        self.weight = weight
    
    def introduce_self(self):
        print(f"Hi, my name is {self.name}")

r1 = Robot("Tom", "red", 30)
r2 = Robot("Jerry", "blue", 40)

class Person:
    def __init__(self, name, personality, is_sitting, robot_owned):
        self.name = name
        self.personality = personality
        self.is_sitting = is_sitting
        self.robot_owned = robot_owned

    def sit_down(self):
        self.is_sitting = True
    
    def stand_up(self):
        self.is_sitting = False

In [15]:
p1 = Person("Alice", "aggressive", False)
p2 = Person("Becky", "talkative", True)

In [16]:
# p1 owns r2
p1.robot_owned = r2
# p2 owns r1
p2.robot_owned = r1

In [18]:
p1.robot_owned.introduce_self()
p2.robot_owned.introduce_self()

Hi, my name is Jerry
Hi, my name is Tom
